In [1]:
import gmsh
import numpy as np

In [2]:
gmsh.initialize()
gmsh.open("basic.msh")

Info    : Reading 'basic.msh'...
Info    : 9 entities
Info    : 160 nodes
Info    : 318 elements
Info    : Storing section $Projection as model attribute
Info    : Done reading 'basic.msh'


In [3]:
# Get nodes
node_tags, node_coords, _ = gmsh.model.mesh.getNodes()

In [4]:
physical_groups = gmsh.model.getEntities()
physical_groups

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 1), (1, 2), (1, 3), (1, 4), (2, 1)]

In [11]:
def getElementToNodeMapping(element_dimension):
    type, tags, nodes = gmsh.model.mesh.getElements(dim=element_dimension)
    result = {}
    num_vertices = element_dimension + 1
    node_tag_offset = 1
    element_tag_offset = min(tags[0])
    for i, tag in enumerate(tags[0]):
        key = tuple(nodes[0][num_vertices * i : num_vertices * (i + 1)] - node_tag_offset)
        result[key] = tag - element_tag_offset
    return result

In [12]:
# Get elements (triangles and edges)
nodes_to_edges = getElementToNodeMapping(element_dimension=1)
nodes_to_triangles = getElementToNodeMapping(element_dimension=2)
nodes_to_edges

{(0, 4): 0,
 (4, 5): 1,
 (5, 6): 2,
 (6, 7): 3,
 (7, 8): 4,
 (8, 9): 5,
 (9, 10): 6,
 (10, 11): 7,
 (11, 12): 8,
 (12, 13): 9,
 (13, 14): 10,
 (14, 15): 11,
 (15, 16): 12,
 (16, 17): 13,
 (17, 18): 14,
 (18, 19): 15,
 (19, 20): 16,
 (20, 21): 17,
 (21, 1): 18,
 (1, 22): 19,
 (22, 23): 20,
 (23, 24): 21,
 (24, 25): 22,
 (25, 26): 23,
 (26, 27): 24,
 (27, 28): 25,
 (28, 29): 26,
 (29, 30): 27,
 (30, 31): 28,
 (31, 2): 29,
 (2, 32): 30,
 (32, 33): 31,
 (33, 34): 32,
 (34, 35): 33,
 (35, 36): 34,
 (36, 37): 35,
 (37, 38): 36,
 (38, 39): 37,
 (39, 40): 38,
 (40, 41): 39,
 (41, 42): 40,
 (42, 43): 41,
 (43, 44): 42,
 (44, 45): 43,
 (45, 46): 44,
 (46, 47): 45,
 (47, 48): 46,
 (48, 49): 47,
 (49, 3): 48,
 (3, 50): 49,
 (50, 51): 50,
 (51, 52): 51,
 (52, 53): 52,
 (53, 54): 53,
 (54, 55): 54,
 (55, 56): 55,
 (56, 57): 56,
 (57, 58): 57,
 (58, 59): 58,
 (59, 0): 59}

In [13]:
nodes_to_triangles = getElementToNodeMapping(element_dimension=2)
nodes_to_triangles

{(98, 108, 109): 0,
 (62, 112, 120): 1,
 (63, 126, 146): 2,
 (99, 84, 145): 3,
 (84, 95, 145): 4,
 (103, 65, 117): 5,
 (111, 113, 144): 6,
 (115, 103, 117): 7,
 (93, 97, 143): 8,
 (105, 61, 107): 9,
 (94, 123, 124): 10,
 (82, 81, 122): 11,
 (70, 92, 125): 12,
 (27, 28, 135): 13,
 (104, 113, 114): 14,
 (133, 76, 141): 15,
 (48, 49, 96): 16,
 (94, 69, 123): 17,
 (61, 105, 109): 18,
 (112, 62, 142): 19,
 (125, 92, 126): 20,
 (71, 81, 82): 21,
 (98, 109, 154): 22,
 (86, 85, 137): 23,
 (84, 99, 154): 24,
 (96, 49, 152): 25,
 (105, 100, 109): 26,
 (82, 103, 116): 27,
 (67, 121, 134): 28,
 (81, 80, 83): 29,
 (103, 115, 116): 30,
 (86, 106, 146): 31,
 (126, 92, 146): 32,
 (94, 124, 159): 33,
 (80, 81, 131): 34,
 (81, 83, 84): 35,
 (81, 71, 131): 36,
 (104, 106, 113): 37,
 (86, 129, 136): 38,
 (116, 68, 127): 39,
 (83, 80, 85): 40,
 (7, 8, 118): 41,
 (35, 36, 123): 42,
 (81, 84, 100): 43,
 (83, 85, 86): 44,
 (74, 91, 157): 45,
 (11, 12, 110): 46,
 (36, 37, 124): 47,
 (44, 45, 117): 48,
 (45, 46

In [34]:
edge_triangles = {}

NameError: name 'array' is not defined

In [25]:
gmsh.model.mesh.getElementEdgeNodes(2)

array([ 99, 109, 109, ..., 158, 158,  92], dtype=uint64)

In [ ]:
for tag, type_code, node_ids in zip(element_tags, element_types, element_nodes):
    if type_code == 2:  # Triangle
        triangles[tag] = node_ids
        for i in range(3):
            edge = tuple(sorted((node_ids[i], node_ids[(i+1)%3])))
            if edge not in edges:
                edges[edge] = []
            edges[edge].append(tag)
    elif type_code == 1:  # Edge
        edge = tuple(node_ids)
        edges[edge] = tag

        # Check if edge is on boundary
        boundary_tag = gmsh.model.mesh.getPhysicalGroupsForEntity(1, tag)
        if boundary_tag:
            if boundary_tag[0] not in boundary_edges:
                boundary_edges[boundary_tag[0]] = []
            boundary_edges[boundary_tag[0]].append(tag)

In [11]:
gmsh.finalize()